In [1]:
import numpy as np 
import pandas as pd 
import urllib.request
from urllib.request import urlretrieve, Request, urlopen
import json
import time
import os
import fnmatch
import re

In [2]:
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 2000)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [3]:
# Concatenating files
!sed 1d Helpmefind_roses*.csv > Helpmefind_roses_merged.csv

In [4]:
df1 = pd.read_csv('Helpmefind_roses_merged.csv')
df1.columns = ["NAME","URL","SYNONYM","ARS","ORIG","CLASS","BLOOM","HABIT","GROWING","PARENTAGE","NOTES"]
df1.head(5)

,NAME,URL,SYNONYM,ARS,ORIG,CLASS,BLOOM,HABIT,GROWING,PARENTAGE,NOTES
0,Evert van Dijk,https://www.helpmefind.com/rose/l.php?l=2.62713.1,• Evert van Dyk,Registration name: Evert van Dyk,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Hybrid Tea.,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Bushy, upright, well-branched. Glossy, dark green foliage.",Hardy.,"\nOphelia (hybrid tea, Paul, 1912) × \nAmerica (hybrid tea, Hill, 1921)","See references for more. The bloom was said to be ""compact"" and the foliage ""reddy green', with the wood and spines ""red""."
1,Evert van Dyk,https://www.helpmefind.com/rose/l.php?l=2.62713,• Evert van Dijk,Registration name: Evert van Dyk,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Hybrid Tea.,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Bushy, upright, well-branched. Glossy, dark green foliage.",Hardy.,"\nOphelia (hybrid tea, Paul, 1912) × \nAmerica (hybrid tea, Hill, 1921)","See references for more. The bloom was said to be ""compact"" and the foliage ""reddy green', with the wood and spines ""red""."
2,Every Good Gift,https://www.helpmefind.com/rose/l.php?l=2.63618,NaN,Medium red Hybrid Tea.Registration name: Every Good GiftExhibition name: Every Good Gift,Bred by Jean Murray (2010).,Hybrid Tea.,"Red. Flowers velvety red. Moderate fragrance. Average diameter 4"". Medium-large, full (26-40 petals), borne mostly solitary bloom form. Blooms in flushes throughout the season.","Medium, upright.",Can be used for cut flower or garden.,"\nNeptune ™ (hybrid tea, Carruth, 2003) × \nLiebeszauber ™ (hybrid tea, Kordes 1991)",
3,Evesafia,https://www.helpmefind.com/rose/l.php?l=2.35646.1,• Saint Fiacre d'Orléans,Orange-pink Polyantha.Registration name: EvesafiaExhibition name: Saint Fiacre d'Orléans,"Bred by André Eve (France, 1997).",Polyantha.,Light pink.,NaN,USDA zone 6b through 9b (default).,"If you know the parentage of this rose, or other details, please contact us.",
4,Evesaine,https://www.helpmefind.com/rose/l.php?l=2.37860.1,• Prestige de Seine-et-Marne ®,Shrub.,"Bred by André Eve (France, 1992).",Shrub.,Butter-yellow.,NaN,USDA zone 6b through 9b (default).,"If you know the parentage of this rose, or other details, please contact us.",


In [5]:
df1['CLASS'].value_counts()

Hybrid Tea.                                                                                      11207
Floribunda.                                                                                      6373 
Shrub.                                                                                           3684 
Miniature.                                                                                       3444 
Hybrid Perpetual.                                                                                2009 
Unknown.                                                                                         1480 
Gallica / Provins.                                                                               1475 
Tea.                                                                                             1319 
Florists Rose, Hybrid Tea.                                                                       1189 
Species / Wild.                                                          

In [6]:
df1['ORIG'].value_counts().head(20)

Bred by André Carl (United States, 2018).                             343
Bred by André Carl (United States, 2013).                             218
Bred by Robert Neil Rippetoe (United States, 2007).                   150
Bred by Unknown (before 1836).                                        139
Bred by Unknown (before 1842).                                        129
Bred by Jacques-Louis Descemet (France, before 1815).                 111
Bred by Unknown English Breeder(s) (United Kingdom, before 1817).     110
Bred by Unknown (before 1846).                                        101
Bred by André Carl (United States).                                   95 
Bred by Unknown.                                                      92 
Bred by Louis Xavier Joseph Miellez (France, before 1828).            92 
Bred by Unknown (before 1820).                                        91 
Bred by Unknown (before 1848).                                        91 
Bred by Unknown (before 1880).        

In [7]:
df1["ORIG"].str.split("(", expand=True)

,0,1,2,3,4,5,6,7
0,Bred by Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None,None,None,None,None
1,Bred by Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None,None,None,None,None
2,Bred by Jean Murray,2010).,None,None,None,None,None,None
3,Bred by André Eve,"France, 1997).",None,None,None,None,None,None
4,Bred by André Eve,"France, 1992).",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
55248,Bred by André Carl,United States).,None,None,None,None,None,None
55249,Bred by André Carl,United States).,None,None,None,None,None,None
55250,Bred by André Carl,United States).,None,None,None,None,None,None
55251,Bred by André Carl,United States).,None,None,None,None,None,None


In [8]:
#df1["ORIG"].str.split("(", expand=True)
orig_split = df1["ORIG"].str.split("(", expand=True)

df1["ORIG1"]= orig_split[0] 
df1["ORIG2"]= orig_split[1] 
df1["ORIG3"]= orig_split[2] 
df1["ORIG4"]= orig_split[3] 

df1[['ORIG','ORIG1','ORIG2','ORIG3','ORIG4']].head(5)

,ORIG,ORIG1,ORIG2,ORIG3,ORIG4
0,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Bred by Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None
1,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Bred by Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None
2,Bred by Jean Murray (2010).,Bred by Jean Murray,2010).,None,None
3,"Bred by André Eve (France, 1997).",Bred by André Eve,"France, 1997).",None,None
4,"Bred by André Eve (France, 1992).",Bred by André Eve,"France, 1992).",None,None


In [9]:
df1['ORIG1'] = df1['ORIG1'].str.replace("Bred","")
df1['ORIG1'] = df1['ORIG1'].str.replace("by ","") 
df1['ORIG1'] = df1['ORIG1'].str.replace("Discovered ","")

In [10]:
df1[['ORIG','ORIG1','ORIG2','ORIG3','ORIG4']].head(100)

,ORIG,ORIG1,ORIG2,ORIG3,ORIG4
0,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None
1,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Gerald Adriaan Van Rossem,"Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",None,None
2,Bred by Jean Murray (2010).,Jean Murray,2010).,None,None
3,"Bred by André Eve (France, 1997).",André Eve,"France, 1997).",None,None
4,"Bred by André Eve (France, 1992).",André Eve,"France, 1992).",None,None
5,"Bred by André Eve (France, circa 1990). Introduced in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.",André Eve,"France, circa 1990). Introduced in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.",None,None
6,"Bred by Jérôme Rateau (France, 2008). Introduced in France by André Eve/Roses Anciennes André Eve in 2013 as 'Axel Kahn'.",Jérôme Rateau,"France, 2008). Introduced in France by André Eve/Roses Anciennes André Eve in 2013 as 'Axel Kahn'.",None,None
7,"Bred by Derrick Everitt (United Kingdom, 2001).",Derrick Everitt,"United Kingdom, 2001).",None,None
8,"Bred by Jérôme Rateau (France, 2009). Introduced in France by André Eve/Roses Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.",Jérôme Rateau,"France, 2009). Introduced in France by André Eve/Roses Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.",None,None
9,"Bred by Jérôme Rateau (France, 2011).",Jérôme Rateau,"France, 2011).",None,None


In [11]:
df1['ORIG1'].value_counts()

 Unknown                                                                             2927
 W. Kordes & Sons                                                                    1192
 Samuel Darragh McGredy IV                                                           1084
 L. Pernille Olesen                                                                  747 
 André Carl                                                                          739 
                                                                                    ...  
Phillip H. Roland                                                                    1   
 Gertrudis Fábregas de Dot                                                           1   
 Aubin Céchet                                                                        1   
Introduced in Germany Weigand in before 1914 as 'Weigand's rostfreie Unterlage'.     1   
 J. Cattel                                                                           1   
Name: ORIG

In [12]:
orig_split = df1["ORIG"].str.split("Introduced", expand=True)

df1["INTRO1"]= orig_split[0] 
df1["INTRODUCED"]= orig_split[1] 
df1["INTRO3"]= orig_split[2] 
df1["INTRO4"]= orig_split[3] 

df1[['ORIG','INTRO1','INTRODUCED','INTRO3','INTRO4']].head(50)

,ORIG,INTRO1,INTRODUCED,INTRO3,INTRO4
0,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.","Bred by Gerald Adriaan Van Rossem (Netherlands, 1931).",in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.,None,None
1,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.","Bred by Gerald Adriaan Van Rossem (Netherlands, 1931).",in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.,None,None
2,Bred by Jean Murray (2010).,Bred by Jean Murray (2010).,None,None,None
3,"Bred by André Eve (France, 1997).","Bred by André Eve (France, 1997).",None,None,None
4,"Bred by André Eve (France, 1992).","Bred by André Eve (France, 1992).",None,None,None
5,"Bred by André Eve (France, circa 1990). Introduced in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.","Bred by André Eve (France, circa 1990).",in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.,None,None
6,"Bred by Jérôme Rateau (France, 2008). Introduced in France by André Eve/Roses Anciennes André Eve in 2013 as 'Axel Kahn'.","Bred by Jérôme Rateau (France, 2008).",in France by André Eve/Roses Anciennes André Eve in 2013 as 'Axel Kahn'.,None,None
7,"Bred by Derrick Everitt (United Kingdom, 2001).","Bred by Derrick Everitt (United Kingdom, 2001).",None,None,None
8,"Bred by Jérôme Rateau (France, 2009). Introduced in France by André Eve/Roses Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.","Bred by Jérôme Rateau (France, 2009).",in France by André Eve/Roses Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.,None,None
9,"Bred by Jérôme Rateau (France, 2011).","Bred by Jérôme Rateau (France, 2011).",None,None,None


In [13]:
del df1['INTRO4']
del df1['INTRO3']
del df1['ORIG2']
del df1['ORIG3']
del df1['ORIG4']

In [14]:
# extracting first year mentioned in the ORIG column
df1['YEAR'] = df1['ORIG'].str.extract('(\\d{4})', expand=False).str.strip()

# extracting pattern 'yyyy - yyyy' mentioned in the ORIG column
df1['YEAR_BRED_BY'] = df1['ORIG'].str.extract('(\\d{4} - \\d{4})', expand=False).str.strip()

# extracting pattern 'yyyy - yyyy' mentioned in the ORIG column
df1['YEAR_BRED_BY2'] = df1['ORIG'].str.extract('(\\d{4}-\\d{4})', expand=False).str.strip()

# printing 50 first rows for modified columns
df1[['ORIG','YEAR', 'YEAR_BRED_BY', 'YEAR_BRED_BY2']].head(50)

,ORIG,YEAR,YEAR_BRED_BY,YEAR_BRED_BY2
0,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",1931,NaN,NaN
1,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",1931,NaN,NaN
2,Bred by Jean Murray (2010).,2010,NaN,NaN
3,"Bred by André Eve (France, 1997).",1997,NaN,NaN
4,"Bred by André Eve (France, 1992).",1992,NaN,NaN
5,"Bred by André Eve (France, circa 1990). Introduced in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.",1990,NaN,NaN
6,"Bred by Jérôme Rateau (France, 2008). Introduced in France by André Eve/Roses Anciennes André Eve in 2013 as 'Axel Kahn'.",2008,NaN,NaN
7,"Bred by Derrick Everitt (United Kingdom, 2001).",2001,NaN,NaN
8,"Bred by Jérôme Rateau (France, 2009). Introduced in France by André Eve/Roses Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.",2009,NaN,NaN
9,"Bred by Jérôme Rateau (France, 2011).",2011,NaN,NaN


Column year not always has year when the rose was created. In some cases first year mentioned in the ORIG column is from the breders date of birth. 

In [15]:
# count of rows where column 'YEAR_BRED_BY' is not null
df1['YEAR_BRED_BY'][(df1['YEAR_BRED_BY'].notnull())].count()

489

In [16]:
df1["ALL_YEARS"] = df1['ORIG'].str.findall("\d+") 

df1[['ORIG','YEAR', 'ALL_YEARS', 'YEAR_BRED_BY']][(df1['YEAR_BRED_BY'].notnull())]

,ORIG,YEAR,ALL_YEARS,YEAR_BRED_BY
25,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1982). Introduced in United Kingdom by Wheatcroft in 1984 as 'Polar Star'.",1912,"[1912, 2006, 1982, 1984]",1912 - 2006
69,"Bred by Mathias Tantau (1882 - 1953) (Germany, 1946).",1882,"[1882, 1953, 1946]",1882 - 1953
531,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1963). Introduced in Australia by Treloar Roses in 1966 as 'Ace of Hearts'.",1912,"[1912, 2006, 1963, 1966]",1912 - 2006
626,"Bred by Anni Berger (1904 - 1990) (Germany, before 1979). Introduced in Germany by GPG Roter Oktober, Bad Langensalza in 1979 as 'Adagio'.",1904,"[1904, 1990, 1979, 1979]",1904 - 1990
827,"Bred by Mathias Tantau (1882 - 1953) (Germany, 1936). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1937 as 'Agnes Kruse'.",1882,"[1882, 1953, 1936, 1937]",1882 - 1953
876,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1964).",1912,"[1912, 2006, 1964]",1912 - 2006
998,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1974).",1912,"[1912, 2006, 1974]",1912 - 2006
1124,"Bred by Francisco Ferrer Marti (1919 - 2004) (Spain, 1991).",1919,"[1919, 2004, 1991]",1919 - 2004
1374,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1958). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1961 as 'Allotria'.",1912,"[1912, 2006, 1958, 1961]",1912 - 2006
1422,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1954). Introduced in United States by Jackson & Perkins Co. in 1954 as 'Alpine Glow'. Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1956 as 'Alpenglühen'.",1912,"[1912, 2006, 1954, 1954, 1956]",1912 - 2006


In [17]:
# In order to correct "YEAR" column where breders dob is taken instead of rose created date 
# we need to replace entry in column "YEAR" when column "YEAR_BRED_BY" is populated
# with the 3rd value in column "YEARS_ALL"

for i, row in df1.iterrows():
    if (pd.notnull(row['YEAR_BRED_BY'])):
        if len(row['ALL_YEARS']) > 2:
            # print (row['YEAR_BRED_BY'])
            row['YEAR'] = row['ALL_YEARS'][2]
        else:
            row['YEAR'] = ''
            
for i, row in df1.iterrows():
    if (pd.notnull(row['YEAR_BRED_BY2'])):
        if len(row['ALL_YEARS']) > 2:
            # print (row['YEAR_BRED_BY'])
            row['YEAR'] = row['ALL_YEARS'][2]
        else:
            row['YEAR'] = ''

df1[['ORIG','YEAR', 'ALL_YEARS', 'YEAR_BRED_BY','YEAR_BRED_BY2']][(df1['YEAR_BRED_BY'].notnull())]

,ORIG,YEAR,ALL_YEARS,YEAR_BRED_BY,YEAR_BRED_BY2
25,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1982). Introduced in United Kingdom by Wheatcroft in 1984 as 'Polar Star'.",1982,"[1912, 2006, 1982, 1984]",1912 - 2006,NaN
69,"Bred by Mathias Tantau (1882 - 1953) (Germany, 1946).",1946,"[1882, 1953, 1946]",1882 - 1953,NaN
531,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1963). Introduced in Australia by Treloar Roses in 1966 as 'Ace of Hearts'.",1963,"[1912, 2006, 1963, 1966]",1912 - 2006,NaN
626,"Bred by Anni Berger (1904 - 1990) (Germany, before 1979). Introduced in Germany by GPG Roter Oktober, Bad Langensalza in 1979 as 'Adagio'.",1979,"[1904, 1990, 1979, 1979]",1904 - 1990,NaN
827,"Bred by Mathias Tantau (1882 - 1953) (Germany, 1936). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1937 as 'Agnes Kruse'.",1936,"[1882, 1953, 1936, 1937]",1882 - 1953,NaN
876,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1964).",1964,"[1912, 2006, 1964]",1912 - 2006,NaN
998,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1974).",1974,"[1912, 2006, 1974]",1912 - 2006,NaN
1124,"Bred by Francisco Ferrer Marti (1919 - 2004) (Spain, 1991).",1991,"[1919, 2004, 1991]",1919 - 2004,NaN
1374,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1958). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1961 as 'Allotria'.",1958,"[1912, 2006, 1958, 1961]",1912 - 2006,NaN
1422,"Bred by Mathias Tantau, Jr. (1912 - 2006) (Germany, 1954). Introduced in United States by Jackson & Perkins Co. in 1954 as 'Alpine Glow'. Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1956 as 'Alpenglühen'.",1954,"[1912, 2006, 1954, 1954, 1956]",1912 - 2006,NaN


In [18]:
df1[['ORIG','YEAR', 'ALL_YEARS', 'YEAR_BRED_BY','YEAR_BRED_BY2']][(df1['YEAR_BRED_BY2'].notnull())].head(50)

,ORIG,YEAR,ALL_YEARS,YEAR_BRED_BY,YEAR_BRED_BY2
28,"Bred by Anne G. Cocker (1920-2014) (Scotland, 2000).",2000,"[1920, 2014, 2000]",NaN,1920-2014
51,"Bred by Gilbert Nabonnand (1828-1903) (France, 1884). Introduced in Australia by C. F. Newman and Sons - Adelaide in 1894 as 'Impératrice Maria Feodorowna de Russie'.",1884,"[1828, 1903, 1884, 1894]",NaN,1828-1903
88,"Bred by Marie-Louise (Louisette) Meilland (1920-1987) (France, 1984).",1984,"[1920, 1987, 1984]",NaN,1920-1987
89,"Bred by Patrick Dickson (1926-2012) (United Kingdom, 1990). Introduced in United Kingdom by Dickson Nurseries Ltd. in 1990 as 'Space Invader'.",1990,"[1926, 2012, 1990, 1990]",NaN,1926-2012
115,"Bred by George Dawson (1904-1991) (Australia, 1973).",1973,"[1904, 1991, 1973]",NaN,1904-1991
123,"Bred by Fred J. Armbrust (1901-1975) (Australia, before 1966). Introduced in Australia by Langbecker in 1966 as 'Eye Liner'.",1966,"[1901, 1975, 1966, 1966]",NaN,1901-1975
226,"Bred by Alister Clark (1864-1949) (Australia, 1941). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1941 as 'A.G. Furness'.",1941,"[1864, 1949, 1941, 1941]",NaN,1864-1949
234,"Bred by Alexander Dickson II (1857-1949) (United Kingdom, 1909). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1912 as 'Alexander Hill Gray'.",1909,"[1857, 1949, 1909, 1912]",NaN,1857-1949
275,"Bred by F. Harmon Saville (1924-1997) (United States, before 1989). Introduced in United States by Nor' East Miniature Roses in 1991 as 'American Rose Centennial'.",1989,"[1924, 1997, 1989, 1991]",NaN,1924-1997
279,"Bred by Hans Jürgen Evers (1940-2007) (Germany, 2001). Introduced in Canada by Eurosa Gardens in 2001 as 'A Royal Bride'. Introduced in Germany by Rosen-Tantau/Tantau Roses in 2001 as 'A Royal Bride'.",2001,"[1940, 2007, 2001, 2001, 2001]",NaN,1940-2007


In [19]:
for i, row in df1.iterrows():
    if (pd.notnull(row['YEAR_BRED_BY2'])):
        row['YEAR_BRED_BY'] = row['YEAR_BRED_BY2']
        
df1[['ORIG','YEAR', 'ALL_YEARS', 'YEAR_BRED_BY','YEAR_BRED_BY2']][(df1['YEAR_BRED_BY2'].notnull())].head(50)

,ORIG,YEAR,ALL_YEARS,YEAR_BRED_BY,YEAR_BRED_BY2
28,"Bred by Anne G. Cocker (1920-2014) (Scotland, 2000).",2000,"[1920, 2014, 2000]",1920-2014,1920-2014
51,"Bred by Gilbert Nabonnand (1828-1903) (France, 1884). Introduced in Australia by C. F. Newman and Sons - Adelaide in 1894 as 'Impératrice Maria Feodorowna de Russie'.",1884,"[1828, 1903, 1884, 1894]",1828-1903,1828-1903
88,"Bred by Marie-Louise (Louisette) Meilland (1920-1987) (France, 1984).",1984,"[1920, 1987, 1984]",1920-1987,1920-1987
89,"Bred by Patrick Dickson (1926-2012) (United Kingdom, 1990). Introduced in United Kingdom by Dickson Nurseries Ltd. in 1990 as 'Space Invader'.",1990,"[1926, 2012, 1990, 1990]",1926-2012,1926-2012
115,"Bred by George Dawson (1904-1991) (Australia, 1973).",1973,"[1904, 1991, 1973]",1904-1991,1904-1991
123,"Bred by Fred J. Armbrust (1901-1975) (Australia, before 1966). Introduced in Australia by Langbecker in 1966 as 'Eye Liner'.",1966,"[1901, 1975, 1966, 1966]",1901-1975,1901-1975
226,"Bred by Alister Clark (1864-1949) (Australia, 1941). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1941 as 'A.G. Furness'.",1941,"[1864, 1949, 1941, 1941]",1864-1949,1864-1949
234,"Bred by Alexander Dickson II (1857-1949) (United Kingdom, 1909). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1912 as 'Alexander Hill Gray'.",1909,"[1857, 1949, 1909, 1912]",1857-1949,1857-1949
275,"Bred by F. Harmon Saville (1924-1997) (United States, before 1989). Introduced in United States by Nor' East Miniature Roses in 1991 as 'American Rose Centennial'.",1989,"[1924, 1997, 1989, 1991]",1924-1997,1924-1997
279,"Bred by Hans Jürgen Evers (1940-2007) (Germany, 2001). Introduced in Canada by Eurosa Gardens in 2001 as 'A Royal Bride'. Introduced in Germany by Rosen-Tantau/Tantau Roses in 2001 as 'A Royal Bride'.",2001,"[1940, 2007, 2001, 2001, 2001]",1940-2007,1940-2007


In [20]:
del df1['YEAR_BRED_BY2']

In [21]:
df1.columns

Index(['NAME', 'URL', 'SYNONYM', 'ARS', 'ORIG', 'CLASS', 'BLOOM', 'HABIT', 'GROWING', 'PARENTAGE', 'NOTES', 'ORIG1', 'INTRO1', 'INTRODUCED', 'YEAR', 'YEAR_BRED_BY', 'ALL_YEARS'], dtype='object')

In [22]:
df1.columns = ['NAME', 'URL', 'SYNONYM', 'ARS', 'ORIG', 'CLASS', 'BLOOM', 'HABIT', 'GROWING', 'PARENTAGE', 'COMMENTS', 'BRED_BY', 'INTRO1', 'INTRODUCED', 'YEAR_CREATED', 'BRED_BY_DOB', 'ALL_YEARS']

In [23]:
df1.head(20)

,NAME,URL,SYNONYM,ARS,ORIG,CLASS,BLOOM,HABIT,GROWING,PARENTAGE,COMMENTS,BRED_BY,INTRO1,INTRODUCED,YEAR_CREATED,BRED_BY_DOB,ALL_YEARS
0,Evert van Dijk,https://www.helpmefind.com/rose/l.php?l=2.62713.1,• Evert van Dyk,Registration name: Evert van Dyk,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Hybrid Tea.,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Bushy, upright, well-branched. Glossy, dark green foliage.",Hardy.,"\nOphelia (hybrid tea, Paul, 1912) × \nAmerica (hybrid tea, Hill, 1921)","See references for more. The bloom was said to be ""compact"" and the foliage ""reddy green', with the wood and spines ""red"".",Gerald Adriaan Van Rossem,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931).",in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.,1931,NaN,"[1931, 1934]"
1,Evert van Dyk,https://www.helpmefind.com/rose/l.php?l=2.62713,• Evert van Dijk,Registration name: Evert van Dyk,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",Hybrid Tea.,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Bushy, upright, well-branched. Glossy, dark green foliage.",Hardy.,"\nOphelia (hybrid tea, Paul, 1912) × \nAmerica (hybrid tea, Hill, 1921)","See references for more. The bloom was said to be ""compact"" and the foliage ""reddy green', with the wood and spines ""red"".",Gerald Adriaan Van Rossem,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931).",in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.,1931,NaN,"[1931, 1934]"
2,Every Good Gift,https://www.helpmefind.com/rose/l.php?l=2.63618,NaN,Medium red Hybrid Tea.Registration name: Every Good GiftExhibition name: Every Good Gift,Bred by Jean Murray (2010).,Hybrid Tea.,"Red. Flowers velvety red. Moderate fragrance. Average diameter 4"". Medium-large, full (26-40 petals), borne mostly solitary bloom form. Blooms in flushes throughout the season.","Medium, upright.",Can be used for cut flower or garden.,"\nNeptune ™ (hybrid tea, Carruth, 2003) × \nLiebeszauber ™ (hybrid tea, Kordes 1991)",,Jean Murray,Bred by Jean Murray (2010).,None,2010,NaN,[2010]
3,Evesafia,https://www.helpmefind.com/rose/l.php?l=2.35646.1,• Saint Fiacre d'Orléans,Orange-pink Polyantha.Registration name: EvesafiaExhibition name: Saint Fiacre d'Orléans,"Bred by André Eve (France, 1997).",Polyantha.,Light pink.,NaN,USDA zone 6b through 9b (default).,"If you know the parentage of this rose, or other details, please contact us.",,André Eve,"Bred by André Eve (France, 1997).",None,1997,NaN,[1997]
4,Evesaine,https://www.helpmefind.com/rose/l.php?l=2.37860.1,• Prestige de Seine-et-Marne ®,Shrub.,"Bred by André Eve (France, 1992).",Shrub.,Butter-yellow.,NaN,USDA zone 6b through 9b (default).,"If you know the parentage of this rose, or other details, please contact us.",,André Eve,"Bred by André Eve (France, 1992).",None,1992,NaN,[1992]
5,EVEsibthea,https://www.helpmefind.com/rose/l.php?l=2.77308.1,• Mademoiselle des Minuits ®,Registration name: Mademoiselle des Minuits ®,"Bred by André Eve (France, circa 1990). Introduced in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.","Hybrid Musk, Rambler.","Crimson, white center, yellow stamens. Small, single (4-8 petals), cluster-flowered bloom form. Prolific, once-blooming spring or summer.",NaN,NaN,"\nSibelius ® (hybrid musk, Lens, 1981)",,André Eve,"Bred by André Eve (France, circa 1990).",in France by André Eve/Roses Anciennes André Eve in 2016 as 'Mademoiselle des Minu

In [28]:
# removing names and other not country entries from orig that are entered in brackets
df1['ORIG'] = df1['ORIG'].str.replace("(père)","")
df1['ORIG'] = df1['ORIG'].str.replace("(fils)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Pere)","")
df1['ORIG'] = df1['ORIG'].str.replace("( & fils)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Simó)","")
df1['ORIG'] = df1['ORIG'].str.replace("( &)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Dee)","")
df1['ORIG'] = df1['ORIG'].str.replace("(aka Schwartz Fils)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Du Pont)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Wholesale)","")
df1['ORIG'] = df1['ORIG'].str.replace("(aka WidowVve)","")
# df1['ORIG'] = df1['ORIG'].str.replace("(S)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Ludwig)","")
df1['ORIG'] = df1['ORIG'].str.replace("(J.H.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Dr.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(-Marsault)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Nikolaus)","")
df1['ORIG'] = df1['ORIG'].str.replace("(vve widow)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Marí)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Only family name known)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Nurseries)","")
df1['ORIG'] = df1['ORIG'].str.replace("(A.J.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Brenham Texas)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Laurie)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Villaresi)","")
df1['ORIG'] = df1['ORIG'].str.replace("(H.M.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(/?)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Gene)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Mrs. Gordon)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Mike)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Dan)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Tosh)","")
df1['ORIG'] = df1['ORIG'].str.replace("(William Edward)","")
df1['ORIG'] = df1['ORIG'].str.replace("(senior)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Alsace)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Bill)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Bert)","")
df1['ORIG'] = df1['ORIG'].str.replace("(aka Pissart)","")
df1['ORIG'] = df1['ORIG'].str.replace("(& Sons)","")
df1['ORIG'] = df1['ORIG'].str.replace("(D.W.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Caroline)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Llorenç)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Retail)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Looijmans)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Grand-Jardin)","")
df1['ORIG'] = df1['ORIG'].str.replace("(C.M. Amling Roses)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Op de Beeck)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Roses)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Ed)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Catalin)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Andrew)","")
df1['ORIG'] = df1['ORIG'].str.replace("(the country)","")
df1['ORIG'] = df1['ORIG'].str.replace("(jardinier)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Réunion)","")
df1['ORIG'] = df1['ORIG'].str.replace("(called Larose)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Star Roses)","")
df1['ORIG'] = df1['ORIG'].str.replace("(post)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Sr.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Гунько)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Arthur William)","")
df1['ORIG'] = df1['ORIG'].str.replace("(of Downley)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Bansal?)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Carlos)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Louisette)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Melle )","")
df1['ORIG'] = df1['ORIG'].str.replace("(Florist's)","")
df1['ORIG'] = df1['ORIG'].str.replace("(California )","")
df1['ORIG'] = df1['ORIG'].str.replace("(not W.C.)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Jimmy)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Dave)","")
df1['ORIG'] = df1['ORIG'].str.replace("(not André )","")  
df1['ORIG'] = df1['ORIG'].str.replace("(vve, widow)","")
df1['ORIG'] = df1['ORIG'].str.replace("(aka Widow,Vve)","")

df1['ORIG'] = df1['ORIG'].str.replace("(Star)","")
df1['ORIG'] = df1['ORIG'].str.replace("(The)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Wal)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Alec)","")
df1['ORIG'] = df1['ORIG'].str.replace("(Brenham Texas)","")
df1['ORIG'] = df1['ORIG'].str.replace("(California)","")              
df1['ORIG'] = df1['ORIG'].str.replace("(not André)","")  
df1['ORIG'] = df1['ORIG'].str.replace("(Melle)","")                   
df1['ORIG'] = df1['ORIG'].str.replace("(Tom)","") 
df1['ORIG'] = df1['ORIG'].str.replace("(Brenham Texas)","") 




# extracting first year mentioned in the ORIG column
df1['COUNTRY'] = df1['ORIG'].str.extract('(\\D+)', expand=False).str.strip()
df1['COUNTRY2'] = df1['ORIG'].str.extract('(\\(.*?)\\)', expand=False).str.strip()
df1['COUNTRY3'] = df1['COUNTRY2'].str.extract('(\\D+)', expand=False).str.strip()


df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("(","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace(",","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("before","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("circa","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("former","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("after","")
df1['COUNTRY3'] = df1['COUNTRY3'].str.replace("United Kingdom ","United Kingdom")



# printing 50 first rows for modified columns
df1[['ORIG','YEAR_CREATED', 'COUNTRY', 'COUNTRY2', 'COUNTRY3']].head(50)

,ORIG,YEAR_CREATED,COUNTRY,COUNTRY2,COUNTRY3
0,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",1931,"Bred by Gerald Adriaan Van Rossem (Netherlands,","(Netherlands, 1931",Netherlands
1,"Bred by Gerald Adriaan Van Rossem (Netherlands, 1931). Introduced in Australia by Hazlewood Bros. Pty. Ltd. in 1934 as 'Evert van Dyk'.",1931,"Bred by Gerald Adriaan Van Rossem (Netherlands,","(Netherlands, 1931",Netherlands
2,Bred by Jean Murray (2010).,2010,Bred by Jean Murray (,(2010,
3,"Bred by André Eve (France, 1997).",1997,"Bred by André Eve (France,","(France, 1997",France
4,"Bred by André Eve (France, 1992).",1992,"Bred by André Eve (France,","(France, 1992",France
5,"Bred by André Eve (France, circa 1990). Introduced in France by André Eve Anciennes André Eve in 2016 as 'Mademoiselle des Minuits'.",1990,"Bred by André Eve (France, circa","(France, circa 1990",France
6,"Bred by Jérôme Rateau (France, 2008). Introduced in France by André Eve Anciennes André Eve in 2013 as 'Axel Kahn'.",2008,"Bred by Jérôme Rateau (France,","(France, 2008",France
7,"Bred by Derrick Everitt (United Kingdom, 2001).",2001,"Bred by Derrick Everitt (United Kingdom,","(United Kingdom, 2001",United Kingdom
8,"Bred by Jérôme Rateau (France, 2009). Introduced in France by André Eve Anciennes André Eve in 2017 as 'Robert Louis Stevenson'.",2009,"Bred by Jérôme Rateau (France,","(France, 2009",France
9,"Bred by Jérôme Rateau (France, 2011).",2011,"Bred by Jérôme Rateau (France,","(France, 2011",France


In [29]:
df1['COUNTRY3'] = df1['COUNTRY3'].str.strip()

In [30]:
df1['COUNTRY3'].value_counts()

                       12693
United States          8653 
France                 8621 
Germany                4141 
United Kingdom         3568 
Netherlands            1686 
Denmark                1476 
Australia              1245 
Canada                 970  
Japan                  964  
New Zealand            772  
India                  656  
Italy                  567  
Belgium                560  
s                      550  
Unknown                398  
Ireland                303  
Luxembourg             279  
South Africa           211  
....-                  206  
Hungary                190  
Switzerland            179  
Czechoslovakia         174  
Finland                155  
Spain                  144  
Scotland               134  
USSR                   121  
Portugal               117  
Angers                 97   
Slovakia               85   
Poland                 83   
.... -                 80   
Czech Republic         62   
China                  62   
c             

In [31]:
#df1['ORIG'][(df1['COUNTRY3'] == 'Russia')]

In [41]:
orig_split = df1["ARS"].str.split("Registration name: ", expand=True)

df1["ARS2"]= orig_split[0] 
df1["REG_NAME"]= orig_split[1] 


df1[['ARS','ARS2','REG_NAME']].head(10)

,ARS,ARS2,REG_NAME
0,Registration name: Evert van Dyk,,Evert van Dyk
1,Registration name: Evert van Dyk,,Evert van Dyk
2,Medium red Hybrid Tea.Registration name: Every Good GiftExhibition name: Every Good Gift,Medium red Hybrid Tea.,Every Good GiftExhibition name: Every Good Gift
3,Orange-pink Polyantha.Registration name: EvesafiaExhibition name: Saint Fiacre d'Orléans,Orange-pink Polyantha.,EvesafiaExhibition name: Saint Fiacre d'Orléans
4,Shrub.,Shrub.,None
5,Registration name: Mademoiselle des Minuits ®,,Mademoiselle des Minuits ®
6,NaN,NaN,NaN
7,Medium yellow Hybrid Tea.Registration name: EVEsilvergoldExhibition name: Noreen Mackey,Medium yellow Hybrid Tea.,EVEsilvergoldExhibition name: Noreen Mackey
8,Registration name: Robert Louis StevensonExhibition name: Robert Louis Stevenson,,Robert Louis StevensonExhibition name: Robert Louis Stevenson
9,Registration name: EvesorjaExhibition name: Génie Leonard ® Evesorja,,EvesorjaExhibition name: Génie Leonard ® Evesorja


In [35]:
orig_split = df1["ARS"].str.split("Exhibition name: ", expand=True)

df1["ARS5"]= orig_split[0] 
df1["EXH_NAME"]= orig_split[1] 


df1[['ARS','ARS5','EXH_NAME']].head(10)

,ARS,ARS5,EXH_NAME
0,Registration name: Evert van Dyk,Registration name: Evert van Dyk,None
1,Registration name: Evert van Dyk,Registration name: Evert van Dyk,None
2,Medium red Hybrid Tea.Registration name: Every Good GiftExhibition name: Every Good Gift,Medium red Hybrid Tea.Registration name: Every Good Gift,Every Good Gift
3,Orange-pink Polyantha.Registration name: EvesafiaExhibition name: Saint Fiacre d'Orléans,Orange-pink Polyantha.Registration name: Evesafia,Saint Fiacre d'Orléans
4,Shrub.,Shrub.,None
5,Registration name: Mademoiselle des Minuits ®,Registration name: Mademoiselle des Minuits ®,None
6,NaN,NaN,NaN
7,Medium yellow Hybrid Tea.Registration name: EVEsilvergoldExhibition name: Noreen Mackey,Medium yellow Hybrid Tea.Registration name: EVEsilvergold,Noreen Mackey
8,Registration name: Robert Louis StevensonExhibition name: Robert Louis Stevenson,Registration name: Robert Louis Stevenson,Robert Louis Stevenson
9,Registration name: EvesorjaExhibition name: Génie Leonard ® Evesorja,Registration name: Evesorja,Génie Leonard ® Evesorja


In [45]:
orig_split = df1["REG_NAME"].str.split("Exhibition name: ", expand=True)

df1["REGI_NAME"]= orig_split[0] 
df1["REGI_NAME2"]= orig_split[1] 


df1[['REG_NAME','REGI_NAME','REGI_NAME2']].head(10)

,REG_NAME,REGI_NAME,REGI_NAME2
0,Evert van Dyk,Evert van Dyk,None
1,Evert van Dyk,Evert van Dyk,None
2,Every Good GiftExhibition name: Every Good Gift,Every Good Gift,Every Good Gift
3,EvesafiaExhibition name: Saint Fiacre d'Orléans,Evesafia,Saint Fiacre d'Orléans
4,None,None,None
5,Mademoiselle des Minuits ®,Mademoiselle des Minuits ®,None
6,NaN,NaN,NaN
7,EVEsilvergoldExhibition name: Noreen Mackey,EVEsilvergold,Noreen Mackey
8,Robert Louis StevensonExhibition name: Robert Louis Stevenson,Robert Louis Stevenson,Robert Louis Stevenson
9,EvesorjaExhibition name: Génie Leonard ® Evesorja,Evesorja,Génie Leonard ® Evesorja


In [48]:
#del df1['ARS2']
#del df1['REGI_NAME2']
#del df1['REG_NAME']
del df1['COUNTRY2']
del df1['COUNTRY']
#df1.head(5)

In [51]:
df1['PETALS'] = df1['BLOOM'].str.extract('(\\(.*?)\\)', expand=False).str.strip()
df1[['BLOOM','PETALS']].head(5)

,BLOOM,PETALS
0,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.",NaN
1,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.",NaN
2,"Red. Flowers velvety red. Moderate fragrance. Average diameter 4"". Medium-large, full (26-40 petals), borne mostly solitary bloom form. Blooms in flushes throughout the season.",(26-40 petals
3,Light pink.,NaN
4,Butter-yellow.,NaN


Not full extract of the tetals data. Will need to think of how to extract data like "20 to 25 petals" and "75 petals".
In some cases petals count varies. Like  - "35 to 40 petals.  Average diameter 2.5".  Medium, double (17-25 petals), full (26-40 petals)"

In [56]:
orig_split = df1["BLOOM"].str.split(".", expand=True)

df1["COLOR"]= orig_split[0] 
df1["COLOR2"]= orig_split[1] 


df1[['BLOOM','COLOR','COLOR2']].head(10)

,BLOOM,COLOR,COLOR2
0,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Carmine-pink, salmon-pink streaks, stripes, flecks","Warm pink, clear carmine pink, rose pink shaded salmon"
1,"Carmine-pink, salmon-pink streaks, stripes, flecks. Warm pink, clear carmine pink, rose pink shaded salmon. Mild fragrance. Large, very double, in small clusters, high-centered bloom form. Blooms in flushes throughout the season.","Carmine-pink, salmon-pink streaks, stripes, flecks","Warm pink, clear carmine pink, rose pink shaded salmon"
2,"Red. Flowers velvety red. Moderate fragrance. Average diameter 4"". Medium-large, full (26-40 petals), borne mostly solitary bloom form. Blooms in flushes throughout the season.",Red,Flowers velvety red
3,Light pink.,Light pink,
4,Butter-yellow.,Butter-yellow,
5,"Crimson, white center, yellow stamens. Small, single (4-8 petals), cluster-flowered bloom form. Prolific, once-blooming spring or summer.","Crimson, white center, yellow stamens","Small, single (4-8 petals), cluster-flowered bloom form"
6,"Red. Medium to large, single (4-8 petals), cluster-flowered bloom form.",Red,"Medium to large, single (4-8 petals), cluster-flowered bloom form"
7,"Yellow. Mild fragrance. 26 to 40 petals. Average diameter 4.75"". Large, full (26-40 petals) bloom form. Blooms in flushes throughout the season.",Yellow,Mild fragrance
8,"Deep pink, ages to lighter . Semi-double (9-16 petals), in large clusters bloom form.","Deep pink, ages to lighter","Semi-double (9-16 petals), in large clusters bloom form"
9,Rose-pink.,Rose-pink,


In [57]:
del df1['COLOR2']

In [59]:
df1['COLOR'].value_counts()

Pink                                                 3675
Light pink                                           2801
Red                                                  2689
Deep pink                                            1772
Dark red                                             1639
                                                     ... 
Light pink, yellow undertones, apricot highlights    1   
Pink blend and light yellow, light yellow center     1   
Purple - pink, crimson shading                       1   
Pink, purple shading, brown highlights               1   
Crimson, cerise shading                              1   
Name: COLOR, Length: 8379, dtype: int64

In [58]:
df1.to_csv('Helpmefind_roses_merged_cleanned.csv')